# Averaged Artifact Substraction with EEG Data
## First step prepare Jupyter Notebook

In [ ]:
import os
from FACET.Facet import Facet
from loguru import logger
import sys

os.getcwd()
os.chdir(os.path.dirname(os.path.abspath("__file__")))

eegDataSet_filepath = "NiazyFMRI.edf"
motiondata_moosmann_filepath = "headmotiondata.tsv"
export_bids_path = "./bids_dir"

#configure logging
logger.remove()
logger.add(sys.stderr, level="INFO")
logger.add("Facet.log", level="DEBUG")

#defining important parameters
window_size = 30 # defines the window size for averaging
upsampling_factor = 10 # defines the resampling factor
artifact_to_trigger_offset_seconds = -0.005 # defines the artifact position in seconds relative to the triggermark.
relative_window_position = -0.5 # defines the relative position of the averaging window over the epochs.
event_id_description_pairs={'trigger':1} # For exporting as bids. It defines the event id and its description.
regex_trigger_annotation_filter = r'\b1\b' # r'\btrigger\b' # Annotations with the description 'trigger' are considered as triggers, declard for find triggers after reimporting from bids
regex_trigger_event_filter = r'\b1\b' # Events with the value 1 are considered as triggers, declared for edf stim channel
unwanted_bad_channels = ['EKG', 'EMG', 'EOG', 'ECG'] # Channels with these names are considered as bad channels and not considered in the processing
evaluation_measures=["SNR", "RMS", "RMS2", "MEDIAN"] # Measures that must be calculated and used for evaluation.

f = Facet()


## Second Step import EEG Data

In [ ]:
f.import_EEG(eegDataSet_filepath, artifact_to_trigger_offset=artifact_to_trigger_offset_seconds, upsampling_factor=upsampling_factor, bads=unwanted_bad_channels)
#Uncomment if you wanna reload as bids. Pro: Annotations will be loaded for better visualization. Cons: Somehow the SNR has a lower value.
#f.export_EEG(path="./export",fmt="bids", event_id=event_id_description_pairs)
#f.import_EEG(path="./export",fmt="bids", artifact_to_trigger_offset=artifact_to_trigger_offset_seconds, bads=unwanted_bad_channels)
f.plot_EEG(title="after import")

## Third Step Preprocessing

In [ ]:
f.get_EEG().mne_raw.crop(0,162) # cropping the data for the niazy dataset, because the edf has bad data at the end
f.pre_processing()


## Fourth Step find triggers

In [ ]:

f.find_triggers(regex_trigger_annotation_filter) # Using Niazys data
f.align_triggers(0) # Aligning the triggers to the first trigger
f.get_analytics().print_analytics()

## Fourth step apply AAS

In [ ]:
f.apply_AAS(window_size=window_size, rel_window_position=relative_window_position)

## Fifth step Remove Artifacts

In [ ]:
f.remove_artifacts( )


## Sixth step Post-Processing

In [ ]:
f.post_processing()

## Seventh display the processed eeg data

In [ ]:
f.plot_EEG(title="after lowpass")

## Eighth Evaluate Results

In [ ]:
eeg_python = f.get_EEG()
f.add_to_evaluate(eeg_python, name="numpy_matrix") # Matrix will result in >1.0 RMS2 because of python mne inaccuracies
logger.info(f.evaluate(measures=evaluation_measures))
f.export_EEG(event_id=event_id_description_pairs, path=export_bids_path, fmt="bids")

